In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
from finvizfinance.screener.overview import Overview
from scipy.stats import linregress
import time

In [ ]:
# ==========================================
# 1.(Finviz Screening)
# ==========================================
def get_filtered_picks(limit_per_country=5000): # Raised limit to capture more
    print("--- STEP 1: Fetching Universe from Finviz ---")
    
    try:
        foverview = Overview()
        
        # Criteria: USA, Strong Buy, High Liquidity, Not Penny Stocks
        filters_dict = {
            'Analyst Recom.': 'Strong Buy (1)',
            'Country': 'USA',
            'Average Volume': 'Over 2M', 
            'Market Cap.': '+Small (over $300mln)'
        }
        foverview.set_filter(filters_dict=filters_dict)
        df_results = foverview.screener_view()
        
        if df_results.empty:
            return pd.DataFrame()
            
        print(f"   Success! Found {len(df_results)} initial candidates.")
        
        # Clean up Column Names immediately
        if 'Analyst Recom' in df_results.columns:
            df_results.rename(columns={'Analyst Recom': 'Recom'}, inplace=True)
            
        # Ensure numeric Price for later
        df_results['Price'] = pd.to_numeric(df_results['Price'], errors='coerce')
        
        return df_results

    except Exception as e:
        print(f"   Error in Finviz Step: {e}")
        return pd.DataFrame()

In [ ]:
# ==========================================
# 2. THE CREDIT MODEL (Z-Score & Margins)
# ==========================================
def calculate_z_score(info, financials, balance_sheet):
    """Calculates Altman Z-Score (Bankruptcy Risk)"""
    try:
        # Extract Key Metrics
        total_assets = balance_sheet.loc['Total Assets'].iloc[0]
        total_liab = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
        current_assets = balance_sheet.loc['Current Assets'].iloc[0]
        current_liab = balance_sheet.loc['Current Liabilities'].iloc[0]
        
        # Z-Score Components
        working_capital = current_assets - current_liab
        retained_earnings = balance_sheet.loc['Retained Earnings'].iloc[0] if 'Retained Earnings' in balance_sheet.index else 0
        
        # Handle EBIT naming differences
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan
            
        market_cap = info.get('marketCap', 0)
        sales = financials.loc['Total Revenue'].iloc[0]

        # Ratios
        A = working_capital / total_assets
        B = retained_earnings / total_assets
        C = ebit / total_assets
        D = market_cap / total_liab
        E = sales / total_assets

        # Formula: 1.2A + 1.4B + 3.3C + 0.6D + 1.0E Altman Z-Score coefficients
        z_score = (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
        return round(z_score, 2)
    except:
        return np.nan

def get_margin_trend(financials):
    """Calculates 3-Year Gross Margin Slope"""
    try:
        years = financials.columns[:3]
        margins = []
        for date in years:
            rev = financials.loc['Total Revenue'][date]
            profit = financials.loc['Gross Profit'][date]
            if rev == 0 or np.isnan(rev): margins.append(0)
            else: margins.append(profit/rev)
            
        # Slope Calculation
        margins = margins[::-1] # Chronological order
        slope, _, _, _, _ = linregress(range(len(margins)), margins)
        
        if slope > 0.005: return "Improving"
        elif slope < -0.005: return "Deteriorating"
        else: return "Stable"
    except:
        return "N/A"


In [ ]:
# ==========================================
# 3. EXECUTION ENGINE (SPLIT BY TIER)
# ==========================================
import time

def run_screener_split():
    # A. Get the List from Step 1
    candidates = get_filtered_picks()
    
    if candidates.empty:
        print("No stocks found to analyze.")
        return None, None, None, None, None

    print(f"\n--- STEP 2: Running Credit Risk Model on {len(candidates)} Stocks ---")
    print("Separating stocks into distinct tiers...")
    
    # Initialize separate lists for each tier
    fortress_data = []
    moonshot_data = []
    distress_data = []
    financial_data = [] # Banks/Insurance (skipped by Z-score)
    middle_data = []    # Everything else
    
    # Loop through ALL candidates
    for index, row in candidates.iterrows():
        ticker = row['Ticker']
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # 1. SEPARATE FINANCIALS
            # Z-Score doesn't work for banks, so we stash them here to analyze later
            sector = info.get('sector', 'Unknown')
            if 'Financial' in sector:
                financial_data.append({
                    'Ticker': ticker,
                    'Sector': sector,
                    'Price': row['Price'],
                    'Recom': row.get('Recom', 'N/A')
                })
                continue
            
            # 2. FETCH DATA
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty:
                continue
            
            # 3. RUN METRICS (Using your helper functions)
            z_score = calculate_z_score(info, fin, bs)
            trend = get_margin_trend(fin)
            
            # Data packet to save
            stock_data = {
                'Ticker': ticker,
                'Company': row.get('Company', 'N/A'),
                'Z-Score': z_score,
                'Margin_Trend': trend,
                'Price': row['Price'],
                'Recom': row.get('Recom', 'N/A'),
                'Sector': sector
            }
            
            # 4. SORT INTO LISTS (The Logic)
            if (z_score > 2.99) and (trend in ["Improving", "Stable"]):
                fortress_data.append(stock_data)
                
            elif (z_score < 1.8) and (trend == "Deteriorating"):
                distress_data.append(stock_data)
                
            elif (z_score < 1.8) and (trend == "Improving"):
                moonshot_data.append(stock_data)
            
            else:
                middle_data.append(stock_data)
            
            # Progress indicator
            if index % 10 == 0:
                print(f"   Processed {index} / {len(candidates)}...", end='\r')

        except Exception:
            continue

    # B. Convert Lists to DataFrames
    fortress_df = pd.DataFrame(fortress_data)
    moonshot_df = pd.DataFrame(moonshot_data)
    distress_df = pd.DataFrame(distress_data)
    financial_df = pd.DataFrame(financial_data)
    middle_df = pd.DataFrame(middle_data)
    
    print(f"\n\n--- ANALYSIS COMPLETE ---")
    print(f"Fortress (Safe):     {len(fortress_df)} stocks")
    print(f"Moonshot (Risky):    {len(moonshot_df)} stocks")
    print(f"Distress (Avoid):    {len(distress_df)} stocks")
    print(f"Financials (Skip):   {len(financial_df)} stocks")
    print(f"Middle of Road:      {len(middle_df)} stocks")
    
    return fortress_df, moonshot_df, distress_df, financial_df, middle_df

# --- RUN IT & UNPACK VARIABLES ---
# This saves the results into 5 separate dataframes immediately
fortress_df, moonshot_df, distress_df, fins_df, mid_df = run_screener_split()
#fins_df are banking and other financial sector stocks cannot use Z score due to customer deposits being liabilities

In [ ]:
if fortress_df is not None and not fortress_df.empty:
    print("--- FORTRESS STOCKS (High Quality) ---")
    # Sort by Z-Score to find the healthiest balance sheets aka safe picks
    display(fortress_df.sort_values(by='Z-Score', ascending=False))

In [ ]:
#high risk picks
if moonshot_df is not None and not moonshot_df.empty:
    print("--- MOONSHOT STOCKS (Speculative) ---")
    # Sort by Price (Cheapest first?) or Recom
    display(moonshot_df.sort_values(by='Price', ascending=True))

In [ ]:
#very high risk / avoid picks
if distress_df is not None and not distress_df.empty:
    print("--- DISTRESS (Avoid / Short Candidates) ---")
    display(distress_df.sort_values(by='Z-Score', ascending=True))